Mount Your Drive to access the DataSet

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install wandb --upgrade

     |████████████████████████████████| 1.7 MB 5.1 MB/s 
     |████████████████████████████████| 144 kB 58.4 MB/s 
     |████████████████████████████████| 181 kB 66.4 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=8e5053a7bddb98d3ebca35c1b678b2796a1875e9b44bf6e8f010cba10fc48fd4
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
#Importing all the packages
from numpy import unique, argmax
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, Activation, Rescaling,BatchNormalization
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import layers
import timeit
from matplotlib import pyplot
import matplotlib.pyplot as plt
import numpy as np
import cv2 
import os
import random
import pickle
from PIL import Image
import pathlib
import wandb

In [ ]:
# We run this to warm up;
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error states that this notebook is not '
      'configured to use a GPU. Change this in the notebook Settings via the Runtime type or  '
      'command palette (cmd/ctrl-shift-P) or the Edit menu .\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

cpu()
gpu()

# Run the op several times.
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))



This error states that this notebook is not configured to use a GPU. Change this in the notebook Settings via the Runtime type or  command palette (cmd/ctrl-shift-P) or the Edit menu .




SystemError: ignored

In [ ]:
#@markdown Mention the path of the train directory
train_dir = "/content/drive/MyDrive/inaturalist_12K/train" #@param {type:"string"}


#@markdown Mention the path of the test directory

val_dir = "/content/drive/MyDrive/inaturalist_12K/val" #@param {type:"string"}

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
imgHeight =  256#@param
imgWidth =  256#@param

imgSize = (imgHeight, imgWidth) 
batchSize = 32 #@param

train_data = tf.keras.utils.image_dataset_from_directory(
                      directory = train_dir,
                      labels = 'inferred',  
                      label_mode = 'categorical',
                      color_mode = 'rgb',
                      batch_size = batchSize,
                      image_size = imgSize,
                      shuffle = True,
                      seed = 5,
                      validation_split = 0.2,
                      subset = 'training')

val_data = tf.keras.utils.image_dataset_from_directory(
                      directory = train_dir,
                      labels = 'inferred',  
                      label_mode = 'categorical',
                      color_mode = 'rgb',
                      batch_size = batchSize,
                      image_size = imgSize,
                      shuffle = True,
                      seed = 5,
                      validation_split = 0.2,
                      subset = 'validation')


# Retaining 10 percent of train and validation data and discarding the rest
# len_train, len_val = len(train_data), len(val_data)
# train_data = train_data.take(int(0.25*len_train))
# val_data = val_data.take(int(0.25*len_val))


train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255,
                validation_split = 0.2,
                shear_range=0.2,
                zoom_range=0.2,
                featurewise_center=False,  # set input mean to 0 over the dataset
                samplewise_center=False,  # set each sample mean to 0
                featurewise_std_normalization=False,  # divide inputs by std of the dataset
                samplewise_std_normalization=False,  # divide each input by its std
                zca_whitening=False,  # apply ZCA whitening
                rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
                width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
                height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
                horizontal_flip=True,  # randomly flip images
                vertical_flip=False
                )


train_data_aug = train_datagen.flow_from_directory(
        train_dir,
        subset='training',
        target_size=imgSize,
        batch_size=batchSize,
        class_mode='categorical',
        shuffle = True,
        seed = 123)

val_data_aug = train_datagen.flow_from_directory(
        train_dir,
        subset='validation',
        target_size=imgSize,
        batch_size=batchSize,
        class_mode='categorical',
        shuffle = True,
        seed = 123)

#Setting it to false to enable testing , Run the next cell to enable it and disable testing. 
isSweep = False


Found 9999 files belonging to 10 classes.
Using 8000 files for training.
Found 9999 files belonging to 10 classes.
Using 1999 files for validation.
Found 8000 images belonging to 10 classes.
Found 1999 images belonging to 10 classes.


Run the below cell to enable Sweep. 

In [ ]:
#@markdown Fill the below form 
isSweep = True #@param {type:"boolean"}

sweepMethod ='bayes' #@param {type:"string"}['grid','bayes','random']

if isSweep:
  sweepConfig = {'name': sweepMethod + '-test-sweep',
                'method': sweepMethod,
                'metric' : {'name': 'val_acc', 
                            'goal': 'maximize'}
                }
  parameters = {
      
      'filterOrganisation' : {'values': [2,1,0.5]},
      'firstLayerFliterCount' : {'values': [64,32]},
      'convLayers' : {'values': [5]},
      'activation' : {'values': ['relu']},
      'denseSize' : {'values': [32,64,128]},
      'optimizer' : {'values': ['adam']},
      'num_epochs' : {'values': [10,20]},
      'kernelSize' : {'values': [3]},
      'dropOut' : {'values': [0.3,0.5]},
      'batchNorm' : {'values': [0,1]},
      'isDataAug' : {'values': [0,1]},
  }

  sweepConfig['parameters'] = parameters
  sweepID = wandb.sweep(sweepConfig, project="cs6910_assignment2", entity="cs6910_assignment")


Create sweep with ID: 0up7ei82
Sweep URL: https://wandb.ai/cs6910_assignment/cs6910_assignment2/sweeps/0up7ei82


In [ ]:
test = []
testname = []
def train(config=False):
  # with wandb.init(config=config):
  # with tf.device('/device:GPU:0'):            
  #Hardcoded for testing 
  filterOrganisation = 0.5 
  firstLayerFliterCount = 64 
  convLayers = 5 
  activation = 'relu'
  denseSize = 32 
  optimizer = 'adam'
  num_epochs = 50
  kernelSize = 3
  isDataAug = 0
  batchNorm=1 
  dropOut =0.2

  #Dynamic values for sweeping
  if isSweep:
    config = sweepConfig
    if config:
      print("Running the Model with the dynamic parameters ")
      print(config)
      wandb.init(config=config)
      config = wandb.init().config
      wandb.run.name = 'firstLayerFliterCount_{}_filter_org_{}_kernelSize_{}_denseSize_{}_dropOut_{}_isDataAug_{}_num_epochs_{}'\
                        .format(config.firstLayerFliterCount, config.filterOrganisation, config.kernelSize,config.denseSize,config.dropOut,config.isDataAug,config.num_epochs)
      filterOrganisation = config.filterOrganisation
      firstLayerFliterCount = config.firstLayerFliterCount
      convLayers = config.convLayers
      activation = config.activation
      denseSize = config.denseSize
      optimizer = config.optimizer
      num_epochs = config.num_epochs
      kernelSize = config.kernelSize
      isDataAug = config.isDataAug
      dropOut = config.dropOut
      test.append(config)
      testname.append(wandb.run.name)

  filterSize = [int(firstLayerFliterCount*(filterOrganisation**layer_num)) for layer_num in range(convLayers)]

  model = Sequential()

  # Applying some convolution and pooling layers
  for layer_num in range(convLayers):
      model.add(Conv2D(filterSize[layer_num], (kernelSize, kernelSize), input_shape=(imgHeight,imgWidth,3)))
      #Normalisation
      if batchNorm:
        model.add(BatchNormalization(axis = -1))
      model.add(Activation('relu'))
      model.add(MaxPool2D(pool_size=(2, 2)))           


  #dense layer
  model.add(Flatten())
  model.add(Dense(denseSize,activation='relu'))

  #Normalisation
  if batchNorm:
    model.add(BatchNormalization(axis = -1))

  #Dropout 
  if dropOut > 0:
    model.add(Dropout(rate = dropOut))

  #output layer
  model.add(Dense(10,activation='softmax'))

  # print(model.summary())

  model.compile(
      optimizer=optimizer,  # Optimizer
      # Loss function to minimize
      loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),#'categorical_crossentropy',
      # List of metrics to monitor
      metrics=['accuracy'],
      )
  
  final_train_data =  train_data_aug if isDataAug else train_data
  final_val_data =  val_data_aug if isDataAug else val_data

  #Hardcoded steps to 15 in order to speed up the training , but this leads to lesser accuracy. use len(train_data_aug) or train_data_aug.samples for maximum accuracy but it is expensive
  esteps =  len(train_data_aug) if isDataAug else len(train_data)
  valSteps = len(val_data_aug) if isDataAug else len(val_data)

  print(test)
  print(testname)
  if isSweep:
    history = model.fit(
                      final_train_data,
                      steps_per_epoch = esteps ,
                      validation_data = final_val_data, 
                      validation_steps = valSteps,
                      epochs = num_epochs,
                      callbacks = [wandb.keras.WandbCallback()]
                      )
  else:
    history = model.fit(
                    final_train_data,
                    steps_per_epoch = esteps ,
                    validation_data = final_val_data, 
                    validation_steps = valSteps,
                    epochs = num_epochs
                    )
    
  #Saving the Model for Future use.   
  if not isSweep:
    model.save(pathlib.Path('/content/drive/MyDrive/DLAssignment2PartAModel'))



In [ ]:
isSweep =True
if isSweep:
  sweepID = 'kkt4yb37'
  wandb.agent(sweepID, function=train)
else:
  train()

wandb: Agent Starting Run: k8zvyx0m with config:
wandb: 	activation: relu
wandb: 	batchNorm: 0
wandb: 	convLayers: 5
wandb: 	denseSize: 128
wandb: 	dropOut: 0.5
wandb: 	filterOrganisation: 0.5
wandb: 	firstLayerFliterCount: 64
wandb: 	isDataAug: 0
wandb: 	kernelSize: 3
wandb: 	num_epochs: 10
wandb: 	optimizer: adam


Running the Model with the dynamic parameters 
{'name': 'bayes-test-sweep', 'method': 'bayes', 'metric': {'name': 'val_acc', 'goal': 'maximize'}, 'parameters': {'filterOrganisation': {'values': [2, 1, 0.5]}, 'firstLayerFliterCount': {'values': [64, 32]}, 'convLayers': {'values': [5]}, 'activation': {'values': ['relu']}, 'denseSize': {'values': [32, 64, 128]}, 'optimizer': {'values': ['adam']}, 'num_epochs': {'values': [10, 20]}, 'kernelSize': {'values': [3]}, 'dropOut': {'values': [0.3, 0.5]}, 'batchNorm': {'values': [0, 1]}, 'isDataAug': {'values': [0, 1]}}}


wandb: Currently logged in as: cs6910_assignment (use `wandb login --relogin` to force relogin)


[{'activation': 'relu', 'batchNorm': 0, 'convLayers': 5, 'denseSize': 128, 'dropOut': 0.5, 'filterOrganisation': 0.5, 'firstLayerFliterCount': 64, 'isDataAug': 0, 'kernelSize': 3, 'num_epochs': 10, 'optimizer': 'adam'}]
['firstLayerFliterCount_64_filter_org_0.5_kernelSize_3_denseSize_128_dropOut_0.5_isDataAug_0_num_epochs_10']
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


250/250 [==============================] - 1749s 7s/step - loss: 2.7703 - accuracy: 0.1493 - val_loss: 2.1956 - val_accuracy: 0.1881 - _timestamp: 1649069512.0000 - _runtime: 1753.0000
Epoch 2/10
250/250 [==============================] - 1583s 6s/step - loss: 2.3431 - accuracy: 0.1866 - val_loss: 2.1849 - val_accuracy: 0.2231 - _timestamp: 1649071095.0000 - _runtime: 3336.0000
Epoch 3/10
250/250 [==============================] - 1579s 6s/step - loss: 2.1715 - accuracy: 0.2240 - val_loss: 2.1593 - val_accuracy: 0.2271 - _timestamp: 1649072674.0000 - _runtime: 4915.0000
Epoch 4/10
250/250 [==============================] - 1598s 6s/step - loss: 2.0733 - accuracy: 0.2570 - val_loss: 2.0428 - val_accuracy: 0.2471 - _timestamp: 1649074272.0000 - _runtime: 6513.0000
Epoch 5/10
250/250 [==============================] - 1602s 6s/step - loss: 2.0265 - accuracy: 0.2666 - val_loss: 2.0856 - val_accuracy: 0.2596 - _timestamp: 1649075918.0000 - _runtime: 8159.0000
Epoch 6/10
250/250 [===========

accuracy,▁▃▄▆▆▇▇███
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▄▃▂▂▂▁▁▁▁
val_accuracy,▁▃▄▅▆▇███▆
val_loss,██▇▄▅▂▂▃▁▆
accuracy,0.31313
best_epoch,8
best_val_loss,1.95117
epoch,9
loss,1.91678
val_accuracy,0.26363


wandb: Agent Starting Run: p18e5z6n with config:
wandb: 	activation: relu
wandb: 	batchNorm: 1
wandb: 	convLayers: 5
wandb: 	denseSize: 64
wandb: 	dropOut: 0.5
wandb: 	filterOrganisation: 1
wandb: 	firstLayerFliterCount: 32
wandb: 	isDataAug: 1
wandb: 	kernelSize: 3
wandb: 	num_epochs: 10
wandb: 	optimizer: adam


Running the Model with the dynamic parameters 
{'name': 'bayes-test-sweep', 'method': 'bayes', 'metric': {'name': 'val_acc', 'goal': 'maximize'}, 'parameters': {'filterOrganisation': {'values': [2, 1, 0.5]}, 'firstLayerFliterCount': {'values': [64, 32]}, 'convLayers': {'values': [5]}, 'activation': {'values': ['relu']}, 'denseSize': {'values': [32, 64, 128]}, 'optimizer': {'values': ['adam']}, 'num_epochs': {'values': [10, 20]}, 'kernelSize': {'values': [3]}, 'dropOut': {'values': [0.3, 0.5]}, 'batchNorm': {'values': [0, 1]}, 'isDataAug': {'values': [0, 1]}}}


[{'activation': 'relu', 'batchNorm': 0, 'convLayers': 5, 'denseSize': 128, 'dropOut': 0.5, 'filterOrganisation': 0.5, 'firstLayerFliterCount': 64, 'isDataAug': 0, 'kernelSize': 3, 'num_epochs': 10, 'optimizer': 'adam'}, {'activation': 'relu', 'batchNorm': 1, 'convLayers': 5, 'denseSize': 64, 'dropOut': 0.5, 'filterOrganisation': 1, 'firstLayerFliterCount': 32, 'isDataAug': 1, 'kernelSize': 3, 'num_epochs': 10, 'optimizer': 'adam'}]
['firstLayerFliterCount_64_filter_org_0.5_kernelSize_3_denseSize_128_dropOut_0.5_isDataAug_0_num_epochs_10', 'firstLayerFliterCount_32_filter_org_1_kernelSize_3_denseSize_64_dropOut_0.5_isDataAug_1_num_epochs_10']
Epoch 1/10
250/250 [==============================] - 1191s 5s/step - loss: 2.5199 - accuracy: 0.1770 - val_loss: 2.4134 - val_accuracy: 0.1101 - _timestamp: 1649085329.0000 - _runtime: 1197.0000
Epoch 2/10
187/250 [=====================>........] - ETA: 4:23 - loss: 2.1650 - accuracy: 0.2410